In [4]:
#以下の手順で必要なライブラリをインストールします。
#
#①まず、chromeのバージョンを調べます。
#　chromeの右上の設定から左一覧の一番下のchromeについて で開いた画面にバージョンが表示されます。
#  2024/5/12現在　124.0.6367.201
#
#②次に、https://pypi.org/project/chromedriver-binary/#history　を開いて、
#　chromedriver-binaryのバージョンの中で、chromeのバージョンを越えない一番最近のバージョン選びます。
#  2024/5/10現在 124.0.6367.201
#
#③Anaconda Prompt（スタートからAnaconda3(64bit) -> Anaconda Prompt　で）を開いて以下のコマンドを実行します。
#
#"pip install selenium"
#"pip install chromedriver-binary==124.0.6367.201"  (==の後に、上記で選んだバージョンを指定します。）
#
#
#まずは、試しに日経プレスリリースから指定したキーワードで検索した結果を確認する。
#指定するキーワードは、"パナソニック AND 発表日：2024"
#
import urllib.parse
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary
# 日経のプレスリリースを確認する
# まずは、日経のプレスリリースのページをブラウザで確認
# https://www.nikkei.com/pressrelease/
#　日経のプレスリリースを検索
# ブラウザーを起動
options = Options()
# 必要ならproxyサーバを設定する
#PROXY = '127.0.0.1:8080'
#options.add_argument('--proxy-server=http://%s' % PROXY)
# ブラウザはヘッドレス（表示しない）とする
options.add_argument('--headless')
# Chromeのドライバを得る
browser = webdriver.Chrome(options=options)

# 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
browser.implicitly_wait(3)

# 検索キーワードをURLエンコードする
keywords = urllib.parse.quote('パナソニック　AND 発表日：2024')
url = 'https://www.nikkei.com/pressrelease/?searchKeyword='+ keywords + '&au=0'

# URLを読み込む
browser.get(url)
# htmlを取得
search_results_html = browser.page_source
print(search_results_html)

<html lang="ja" data-service-name="k2-stream" data-is-webview="false"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1"><meta name="theme-color" content="#003e70"><title>プレスリリースのニュース一覧 - 日本経済新聞</title><meta property="og:title" content="プレスリリースのニュース一覧 - 日本経済新聞"><meta property="twitter:title" content="プレスリリースのニュース一覧"><meta property="og:site_name" content="日本経済新聞"><meta property="og:locale" content="ja_JP"><meta name="description" content="【日経】プレスリリースのニュースをお届けします。"><meta property="og:description" content="【日経】プレスリリースのニュースをお届けします。"><meta name="x-f1h-custom-value" content="{&quot;DSR&quot;:1,&quot;dp&quot;:&quot;r&quot;,&quot;TOPIC&quot;:[&quot;DF_PR_________&quot;,&quot;sokuho&quot;]}"><style type="text/css" nonce="">.ad_content_mask, .ad-slot-234-60, .publisher_ad, .banner-ad-inner, .ad-transition, #cookie_consent_container, #Adv8, #Adv9, .gb_area_ads, .msgad, .inlinead-tagtop, .follower-ad-bottom, .adsense-header, .hotel-ad

In [5]:
# 記事本体の確認
url = "https://www.nikkei.com/article/DGXZRSP671081_Y4A500C2000000/"
# URLを読み込む
browser.get(url)
# htmlを取得
search_results_html = browser.page_source
print(search_results_html)

<html lang="ja" data-service-name="kite-article" data-is-webview="false"><head><meta charset="utf-8"><meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1"><style type="text/css" nonce="">.ad_content_mask, .ad-slot-234-60, .publisher_ad, .banner-ad-inner, .ad-transition, #cookie_consent_container, #Adv8, #Adv9, .gb_area_ads, .msgad, .inlinead-tagtop, .follower-ad-bottom, .adsense-header, .hotel-ad, .spon-img[src*=".alicdn."], .jw-ad-label, .lj-like-item, #intro_ad_1, .a-icon-share-pinterest, #ad-mid-rect, #ad-base, #homepage_top_ads, #left_ads, #left_adv, .Sailthru_Subscribe, .ad-vertical, #eucookienotice, #leaderAdContainer, .nav-ads, #kopf-werbung, .js-ad_iframe, .ad_940_0, #infos_cookie, .grey-ad-notice, .navbar-header-ad, .doubleClickAd, .mb-list-ad, #advertising_wrapper, .etn-ad-text, #adsDisplay, #AdSpaceLeaderboard, a[href*=".wzfqitm.com/"], #adspace_header, .adz-horiz, #inArticleAdv, .ad-hoverable, .buySellAdsContainer, .ac_adbox, #taboola-below-

In [6]:
# プレスリリースのスクレイピング
import sys
import urllib.parse
import re
from html import unescape
import datetime
import os

from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import chromedriver_binary
from bs4 import BeautifulSoup

def get_article_to_text(search_results, file):
  old_url=''
  for article_local_url in re.findall(r'<a href="(.*?)"', search_results, re.DOTALL):
    if article_local_url[0:6]=="/press":
        continue
    if article_local_url[0:5]=="https":
        continue
    if article_local_url==old_url:
        old_url=''
        continue    
    old_url=article_local_url
    article_url = 'https://www.nikkei.com'+ article_local_url
    print(article_url)
    # 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
    browser.implicitly_wait(3)
    # URLを読み込む
    browser.get(article_url)
    # htmlを取得
    article_html = browser.page_source
    #print(article_html)
    header = re.search(r'<h1 class="title_t1xgcrem">(.*?)</h1>',article_html,re.DOTALL).group(1)
    date = re.search(r'<time datetime=(.*?)000Z">(.*?)</time>',article_html,re.DOTALL).group(2)
    print(header,date)
    # 記事本体からテキストのみを取り出し、テキストファイルに書き出す 
    a = re.search(r'発表日:(.*?)すべての記事が読み放題',article_html,re.DOTALL).group(1)
    article = re.search(r'<p class="paragraph_p18mfke4">(.*?)アプリで開く', a, re.DOTALL).group(1)
    article = re.sub('<.*?>', '', article)  # 不要なタグをすべて取り除く
    try:
      file.write(article)
      file.write('\n')
    except:
      print(article)
  return

# 企業名　と　発表年　で日経のプレスリリースを検索するURLを作成
# 検索キーワードをURLエンコードする
keywords = urllib.parse.quote('パナソニック AND 発表日：2024')
url = 'https://www.nikkei.com/pressrelease/?searchKeyword='+ keywords + '&au=0'

#　日経のプレスリリースを検索
# ブラウザーを起動
options = Options()
# 必要ならproxyサーバを設定する
#PROXY = '127.0.0.1:8080'
#options.add_argument('--proxy-server=http://%s' % PROXY)
# ブラウザはヘッドレス（表示しない）とする
options.add_argument('--headless')
# Chromeのドライバを得る
browser = webdriver.Chrome(options=options)

# 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
browser.implicitly_wait(3)
# URLを読み込む
browser.get(url)
# htmlを取得
search_results_html = browser.page_source
# 検索結果の件数からページ数を得る（検索結果は１ページあたり２０件までのため）
results_num = re.search(r'<div aria-label="page counter".*?<span class="total.*?>(.*?)<span class="unit.*?</div>',search_results_html,re.DOTALL).group(1)
print(results_num)
#pages_num = -(-(int(results_num))//20)
pages_num = 1  # 検索結果のはじめの１ページ（２０件）分のみに制限

file = open(urllib.parse.unquote(keywords) + 'articles.txt', 'a', encoding='UTF-8')

search_results = re.search(r'<div><time class="dateHeadline(.*?)<p class',search_results_html,re.DOTALL).group(1)
#print(search_results)
get_article_to_text(search_results, file)

for i in range(pages_num-1):
  url = 'https://www.nikkei.com/pressrelease/?searchKeyword='+ keywords + '&page='+ str(i+2)
  # 暗黙的な待機を最大3秒行う(サーバーの負担軽減)
  browser.implicitly_wait(3)
  # URLを読み込む
  browser.get(url)
  # htmlを取得
  search_results_html = browser.page_source

  search_results = re.search(r'<div><time class="dateHeadline(.*?)<p class',search_results_html,re.DOTALL).group(1)
    
  get_article_to_text(search_results, file)

# ブラウザを終了
browser.quit()
file.close()

81
https://www.nikkei.com/article/DGXZRSP671081_Y4A500C2000000/
両備システムズとパナソニック コネクト、多要素認証ソリューションで共創を開始 2024年5月8日 11:37
https://www.nikkei.com/article/DGXZRSP670768_U4A420C2000000/
日本ストラタステクノロジー、次世代無停止型プラットフォーム「ztC Endurance」を販売開始 2024年4月24日 11:01
https://www.nikkei.com/article/DGXZRSP670721_T20C24A4000000/
JEITA、3月のパーソナルコンピュータ国内出荷実績を発表 2024年4月23日 11:27
https://www.nikkei.com/article/DGXZRSP670708_S4A420C2000000/
パナソニック ハウジングソリューションズ、「間取り図AI積算」を開始 2024年4月22日 17:01
https://www.nikkei.com/article/DGXZRSP670665_S4A420C2000000/
パナソニック エナジー、次世代の蓄電池人材育成に向けて「MIRAI奨学金」を新設し募集を開始 2024年4月22日 11:01
https://www.nikkei.com/article/DGXZRSP670644_Z10C24A4000000/
日産自とパナソニック、「NissanConnect」と「音声プッシュ通知」を連携したサービスを開始 2024年4月19日 15:01
https://www.nikkei.com/article/DGXZRSP670480_W4A410C2000000/
パナソニック ソリューションテクノロジー、法人向けeラーニング講座サービスを提供開始 2024年4月16日 11:17
https://www.nikkei.com/article/DGXZRSP670457_V10C24A4000000/
パナソニック、エレクトリックワークス社が地域情報コミュニケーションサービス「まちベル」を提供開始 2024年4月15日 15:45
https://www.nikkei.com/article/